# Phase Transitions in Magnets
### Christina Lee
### Category: Prerequisites
### Prerequisites: Monte Carlo Ferromagnet, Statistical Mechanics

In Monte Carlo Ferromagnets, we only looked at the state of a magnet at one temperature.  While you could go back through and evaluate the code at any temperature, and I displayed how it looked at both a high temperature and a low temperature situation, that all had to be done manually.  Today we are going to automate our study of temperature's effect.  

First off, what's the theory?  

The standard aparatus for phase transitions is <b>Landau Theory</b>.  In Landau theory, we assume that we have an <b>order parameter</b>. 

Order parameters have
* local physical meaning
* global observational consequences

For example, in magnets we use the local magnetization $m$: in freezing fluids, density $\rho$.  The value of an order parameter should undergo a jump across a phase transition, either from a zero to a non-zero value or from one value to another.  

Non-zero order parameters also represent symmetry breaking.  In an unmagnetized magnet, all directions are equivalent, but once it magnetizies, it chooses a preferred direction.  That direction is arbitrary up until the magnet spotaneously chooses it.  

![Symmetry Breaking](Images/PhaseTransitions/symmetrybreaking.svg)

For certain Chinese styles of meals, many dishes are brought out for the entire table, and you serve yourself from the dishes in the center.  To make serving easier, sometimes the table might have a platform in the middle you can rotate.  Nothing dictates whether the table must go clockwise or counterclockwise, but once someone decides the platform is going one way, that's usually the way it continues for the rest of the night.  The same could be said for ballroom dancers or cars going down the right (or left) side of the road.  



### Specific Heat and Magnetic Susceptibility

In addition to temperature dependence, today I will also be introducing the physical observables specific heat and magnetic susceptibility.  We do calculate these based on numbers calculated during a specific run, but they also describe how the numbers change with temperature.

Let's take a look at the first one, <b>Specific Heat</b>:
\begin{equation}
C_h=\frac{d \langle E \rangle}{dT}
\end{equation}
While this is the formal definition, the more convenient expression can be calculated by plugging in the Maxwell-Boltzmann Distribution:

\begin{equation}
\frac{ d
\frac{\sum_{i} E_{i} e^{-\beta E_i}}{ \sum_j e^{-\beta E_j} }
}{dT}=\beta^2 \left( \langle E^2 \rangle - \langle E \rangle ^2 \right)
\end{equation}
This is the one we will be using to calculate the specific heat.  For each temperature, we will then need to then calculate both the expecation of the Energy, and the expectation of the Energy Squared.

While in general a material may be difficult to describe, near phase transitions several properties diverge according to corresponding <b>critical exponents</b>.  These critical exponents obey <b>universality</b>.  Phase transitions in the same class will posses the same critical exponents.  Determining the exponent for the 2D square lattice through our simulations also determines the exponent for situations like the mixing of two materials or the percolation problem.

We define our specific heat critical exponent this way,
\begin{equation}
C_h \propto \left( T- T_c \right)^{-\alpha_{\pm}}.
\end{equation}

That works for a large bulk material, but for our simulations, the correlation length, $\xi$, exceeds the size of the system.  This distorts its ability to achieve a proper divergence, since the divergence of the correlation length $\xi$ underpins the divergence of all other quantities.  Therefore we modify our equation to the following form,
\begin{equation}
C_h \propto \left( T - T_c \right)^{-\alpha_{\pm}} g\left( \frac{L}{\xi (T)} \right).
\end{equation}


\begin{equation}
\chi =\frac{d \langle M \rangle}{dT}=\beta^2 \left( \langle M^2 \rangle - \langle M \rangle ^2 \right)
\end{equation}

In [2]:
push!(LOAD_PATH,".")
using Lattices;
using PyPlot

Instead of going into calculating all the lattice parameters again, we will use a class I define in the file Lattices.jl .  This class contains

Lattice Types
* Chain
* Square
* Honeycomb
You can edit the file to make your own types.

Once a lattice is created, it contains <b>Members of Type</b>:
* `name`, a string
* `l`, length in number of unit cells
* `dim`, dimension of lattice
* `a`, array containing the basis vectors by which positions are generated
* `unit`, array of positions inside a single unit
* `N`, number of total sites
* `X`, array of positions
* `nnei`, number of nearest neighbors
* `neigh`, Array of nearest neighbors [i][j], where i is site and j is 1:nnei

Today, I will just look at the square lattice, since that indicates much of the standard phase transition properties.  Some of the lattices I have shown (kagome, triangular, ...) are special frustrated lattices, and thus will behave very wierdly in this situation.

In [4]:
## Define l here
l=20;

lt=MakeLattice("Checkerboard",l);
S=ones(Int8,l,l);  #Our spins
dt=1/(lt.N);

In [5]:
# The energy contribution of just one site
function dE(i::Int)
    Eii=0;
    for j in 1:lt.nnei
        Eii+=S[lt.neigh[i,j]];
    end
    Eii*=-J*S[i];  # we are computing J sz_i sz_j for one i
    return Eii;
end
# The energy of the entire lattice
function E()
    Evar=0;
    for k in 1:lt.N
        Evar+=.5*dE(k);
    end
    return Evar;
end
# The magnetization of the entire lattice
function M()
    Mvar=0;
    for k in 1:lt.N
        Mvar+=S[k];
    end
    return Mvar;
end
"defined functions"

"defined functions"

# Adjustable Parameters

I have set up the simulation so that you can perform two different things.  For one, you can set `video=true` and `t` to a small variable.  Then in a new window you see what the configuration looks like each time you measure.  

Or you can set `video=false` and `t` to a large variable, and actually measure the statistics for the system over a bunch of configurations. 

In [16]:
beta0=.1;
betaf=2;
dbeta=.1;

J=-1;
t=10000;
nskip=10;   # don't measure every sweep= better decorrelation
"Parameters set"

In [17]:
nmeas=Int64(t/nskip); # how many times we will measure
betas=collect(beta0:dbeta:betaf)
"done"

In [18]:
function MCMCMagnet(beta::Float64)
    tm=1; #Our measurement time step
    
    Ma=Array{Int32}(nmeas); # our magnetization measurements
    Ea=Array{Int32}(nmeas); # our energy measurements
    Ma2=Array{Int32}(nmeas); # magnetization squared
    Ea2=Array{Int32}(nmeas); # energy squared
    
    for ti in 1:t
        for j in 1:lt.N
            i = rand(1:lt.N); #Choosing a random site
            de=dE(i);
            if(de>0 || rand()<exp(2*beta*de) ) 
                S[i]=-S[i]; #Switch the sign
            end
        end
        if isapprox(mod(ti,nskip),0)
            Ma[tm]=M();
            Ma2[tm]=Ma[tm]^2;
            
            Ea[tm]=E();
            Ea2[tm]=Ea[tm]^2;
            
            tm+=1;

        end
    end
    Mave=mean(Ma);
    Mstd=std(Ma)/lt.N;
    Eave=mean(Ea);
    Estd=std(Ea)/lt.N;
    
    E2ave=mean(Ea2);
    M2ave=mean(Ma2);
    
    Ch=beta^2*(E2ave-Eave^2)/lt.N;
    χ=beta*(M2ave-Mave^2)/lt.N;
    
    return Mave/lt.N,Mstd,Eave/lt.N,Estd,Ch,χ
end

MCMCMagnet (generic function with 1 method)

In [19]:
Mm=zeros(betas)
Mstd=zeros(betas)
Ee=zeros(betas)
Estd=zeros(betas)
Ch=zeros(betas)
χ=zeros(betas)
for ii in 1:length(betas)
    Mm[ii],Mstd[ii],Ee[ii],Estd[ii],Ch[ii],χ[ii]=MCMCMagnet(betas[ii])
    println("beta: ",betas[ii],"\tM: ",Mm[ii],"\tE: ",Ee[ii])
end

beta: 0.1	M: -0.000985	E: -0.24251
beta: 0.2	M: 0.00134	E: -0.40874000000000005
beta: 0.3	M: -0.00083	E: -0.52148
beta: 0.4	M: 0.0007199999999999999	E: -0.603
beta: 0.5	M: 0.000935	E: -0.66281
beta: 0.6	M: 0.00025	E: -0.71327
beta: 0.7	M: -3.0e-5	E: -0.75657
beta: 0.8	M: 0.000625	E: -0.7917400000000001
beta: 0.9	M: -0.000235	E: -0.82604
beta: 1.0	M: -0.00048	E: -0.85421
beta: 1.1	M: 0.00010999999999999999	E: -0.8771800000000001
beta: 1.2	M: 0.00018999999999999998	E: -0.89866
beta: 1.3	M: -9.499999999999999e-5	E: -0.9167400000000001
beta: 1.4	M: 9.499999999999999e-5	E: -0.93147
beta: 1.5	M: 0.00021	E: -0.94379
beta: 1.6	M: 7.000000000000001e-5	E: -0.955
beta: 1.7	M: 2.0e-5	E: -0.9626600000000001
beta: 1.8	M: 0.000305	E: -0.97016
beta: 1.9	M: -2.5e-5	E: -0.97665
beta: 2.0	M: 8.999999999999999e-5	E: -0.9808100000000001


In [ ]:
#writedlm("PhaseTransitionsL20T50000.dat",[betas Mm Mstd Ee Estd Ch χ])

In [ ]:
temp=readdlm("PhaseTransitionsL50T50000.dat")
Mm50=temp[:,2];
Mstd50=temp[:,3];
Ee50=temp[:,4];
Estd50=temp[:,5];
Ch50=temp[:,6];
χ50=temp[:,7];

In [ ]:
#Mm20=Mm
#Ee20=Ee
#Ch20=Ch
#χ20=χ;
#Mm10=Mm
#Ee10=Ee
#Ch10=Ch
#χ10=χ;
#Mm50=Mm;
#Ee50=Ee;
#Ch50=Ch;
#χ50=χ;
Mm20=Mm;
Ee20=Ee;
Ch20=Ch;
χ20=χ;

In [ ]:
plot(1./(betas),Ch)

In [ ]:
plot(betas,abs(Mm10),label="l=10")
plot(betas,abs(Mm20),label="l=20")
plot(betas,abs(Mm50),label="l=50")
xlabel("Inverse Temperature β")
ylabel("Magnetization")
legend()
title("2D Square Ferromagnet Phase Transition")

In [ ]:
plot(betas,Ee10,label="l=10")
plot(betas,Ee20,label="l=20")
plot(betas,Ee50,label="l=50")
xlabel("Inverse Temperature β")
ylabel("Magnetization")
legend()
title("2D Square Ferromagnet Phase Transition")

In [ ]:
plot(betas,Ch10,label="l=10")
plot(betas,Ch20,label="l=20")
plot(betas,Ch50,label="l=50")
xlabel("Inverse Temperature β")
ylabel("Specific Heat")
legend()
title("Specific Heat Divergence with Finite Size Scaling")

I presented two different equivalent formulas for the specific heat.  Let's check to make sure their equivalent.  

For the formal defination, we'll take a numeric derivative with respect to $\beta$,
\begin{equation}
\frac{d \langle E \rangle}{dT} \approx \frac{ \Delta \langle E \rangle}{\Delta \beta}\frac{d \beta}{d T}
\end{equation}

Note: I corrected the scaling by hand.

In [ ]:
Ch2=(Ee50[1:end-1]-Ee50[2:end]).*betas[2:end].^2

plot(betas[2:end],Ch2*35,label="dE/dT")
plot(betas,Ch50,label="beta2(<E2>-<E>2)")
xlabel("Temperature")
ylabel("Specific Heat")
legend()
title("Comparing Specific Heat Calculations")


In [ ]:
plot(betas,χ10,label="l=10")
plot(betas,χ20,label="l=20")
plot(betas,χ50,label="l=50")
xlabel("Inverse Temperature β")
ylabel("Susceptibility")
legend()
title("Magnetic Susceptibility with Finite Size Scaling")

In [ ]:
M2=(Mm50[1:end-1]-Mm50[2:end]).*(betas[1:end-1]).^2
plot(betas[1:end-1],M2*750,label="dM/dT")
plot(betas,χ50,label="(<M2>-<M>2)beta2")
legend()
xlabel("Inverse Temperature β")
ylabel("Specific Heat")
title("Comparing Methods of Computing Susceptibility")